In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
import cv2

import skimage.measure

import rasterio
from rasterio.features import shapes

import matplotlib.patches as mpatches
from shapely.geometry import Point, Polygon, shape, mapping
import shapely
import geopandas as gpd

from matplotlib.path import Path
import laspy
import open3d as o3d


In [ ]:
image = cv2.imread('/home/frederik/data/TestData/meters_idw.tif', cv2.IMREAD_UNCHANGED)
image = np.where(image >= 0, image, 0)
image = image/np.max(image)

image = (image*255).astype(np.uint8)
plt.title("Original Image")
plt.imshow(image, cmap='gray')
plt.show()

kernel = np.ones((70,70),np.uint8)
closing = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)

plt.title("Closing")
plt.imshow(closing, cmap='gray')
plt.show()


#max_pooling = skimage.measure.block_reduce(closing, (10,10), np.max)

#plt.title("Max Pooling")
#plt.imshow(max_pooling, cmap='gray')
#plt.show()


# Apply edge detection method on the image
edges = cv2.Canny(closing, 4, 160, None, 3)

plt.title("Edges")
plt.imshow(edges, cmap='gray')
plt.show()

# Probabilistic Line Transform
# min_line_length, max_line_gap

linesP = cv2.HoughLinesP(edges, 1, np.pi / 180, 100)

lines_image = np.zeros_like(image)
# Draw the lines
if linesP is not None:
    for i in range(0, len(linesP)):
        l = linesP[i][0]
        cv2.line(lines_image, (l[0], l[1]), (l[2], l[3]), (255,0,0), 3)

plt.title("Hough Lines")
plt.imshow(lines_image, cmap='gray')
plt.show()

kernel = np.ones((5,5),np.uint8)
dilation = cv2.dilate(lines_image, kernel, iterations = 7)
plt.imshow(dilation, cmap='gray')
plt.title("Dialation")
plt.show()


# Pixels per kilometer
x_pixels, y_pixels = image.shape

# Pixels per meter
x_pixels, y_pixels = x_pixels/1000, y_pixels/1000

# Set kernel size to 1 meter around the each line
kernel_size = int(2*np.ceil(x_pixels))*2

# Create kernel
circular_kernel = np.zeros((kernel_size, kernel_size), np.uint8)

# Create a cirkular kernel using (image, center_coordinates, radius, color, thickness)
cv2.circle(circular_kernel, (int(kernel_size/2), int(kernel_size/2)), int(kernel_size/2), 255, -1)

# Perform dilation with the cirkular kernel
dilation_cirkular_kernel = cv2.dilate(dilation, circular_kernel, iterations=1)

plt.title("Dilation Cirkular Kernel")
plt.imshow(dilation_cirkular_kernel, cmap="gray")
plt.show()

In [ ]:
# gdal_polygonize -f gpkg small_area.tif small.gpkg

In [ ]:
# Flipped Polygons

mask = (dilation_cirkular_kernel == 0)
output = rasterio.features.shapes(255-dilation_cirkular_kernel, mask=mask, connectivity=4)
multipolygons_list = list(output)
print("Amount of multipolygons: ", len(multipolygons_list))

polygons = []
for multi_polygons in multipolygons_list:
    for polygon in multi_polygons[0]['coordinates']:
        polygons.append(Path(polygon))

print("Amount of polygons: ", len(polygons))

In [ ]:
mask = (dilation_cirkular_kernel == 255)
output = rasterio.features.shapes(dilation_cirkular_kernel, mask=mask, connectivity=4)
multipolygons_list = list(output)
print("Amount of multipolygons: ", len(multipolygons_list))

polygons = []
for multi_polygons in multipolygons_list:
    for polygon in multi_polygons[0]['coordinates']:
        polygons.append(Path(polygon))

print("Amount of polygons: ", len(polygons))

In [ ]:
plt.figure()
plt.imshow(dilation_cirkular_kernel, cmap='gray')

for i in polygons:
    x = i.vertices[:,0]
    y = i.vertices[:,1]
    plt.scatter(x,y, s=0.001, color='red')
    
plt.title("Polygons")
plt.show()

In [ ]:
polygons = []
for multi_polygons in multipolygons_list:
    for polygon in multi_polygons[0]['coordinates']:
        polygons.append(Polygon(polygon))

print("Amount of polygons: ", len(polygons))

simplified_polygons = []
for p in polygons:
    simplified_polygons.append(shapely.simplify(p, tolerance=4, preserve_topology=True))
    
simplified_polygons  = [p for p in simplified_polygons if not p.is_empty]

print("Amount of simplified polygons: ", len(simplified_polygons))
    
plt.figure()
plt.imshow(dilation_cirkular_kernel, cmap='gray')

for p in simplified_polygons:
    x, y = p.exterior.coords.xy
    plt.scatter(x,y, s=0.01, color='red')
    
plt.title("Simplified Polygons")
plt.show()



In [ ]:
# Pixels per kilometer
x_pixels, y_pixels = image.shape

def MaxMinNormalize(arr):
    return (arr - np.min(arr))/(np.max(arr)-np.min(arr))

def CastAllXValuesToImage(arr, x_pixels):
    return MaxMinNormalize(arr)*x_pixels

def CastAllYValuesToImage(arr, y_pixels):
    return (1-MaxMinNormalize(arr))*y_pixels

In [ ]:
# Create Path Polygons
bbox_polygon = [p.bounds for p in polygons]

simplified_path_polygons = []
bbox_path_polygons = []

for i in range(len(bbox_polygon)):
    
    tempBox = bbox_polygon[i]
    #(minx, miny, maxx, maxy)
    x_min = tempBox[0]
    x_max = tempBox[2]
    
    y_min = tempBox[1]
    y_max = tempBox[3]
    
    bb = [(x_min, y_min), (x_min, y_max), (x_max, y_max), (x_max, y_min)]
    bbox_path_polygons.append(Path(bb))
    
    tempPolygon = simplified_polygons[i]
    simplified_path_polygons.append(Path(mapping(simplified_polygons[i])['coordinates'][0]))

In [ ]:
las = laspy.read(r'/home/frederik/data/denmark/raw/train/PUNKTSKY_00005_1km_6090_503.laz')

In [ ]:
x_values = CastAllXValuesToImage(las.X, x_pixels)
y_values = CastAllXValuesToImage(las.Y, x_pixels)

# Format: [(1,1), (3,5), (1,5), ...] with 30 mio samples
list_zipped = np.array(list(zip(x_values,y_values)))

# Mask to obtain the final indexes of the las file
#indexes_needed_iter1 = np.zeros(len(x_values), dtype=bool)

indexes_needed = np.zeros(len(x_values), dtype=bool)

print(len(bbox_path_polygons))

# Run through all polygons and check which points are inside the polygon
for i in range(len(bbox_path_polygons)):
    #indexes_in_bb = bbox_path_polygons[i].contains_points(list_zipped)
    #indexes_needed = indexes_needed | indexes_in_bb
    #coordinates_in_bb = list_zipped[indexes_in_bb]
    
    indexes_in_polygon = simplified_path_polygons[i].contains_points(list_zipped)
    indexes_needed = indexes_needed | indexes_in_polygon
    print(i)


In [ ]:
print("Point Cloud points: ", np.sum(indexes_needed))
new_las = las[indexes_needed]

point_data = np.stack([new_las.X, new_las.Y, new_las.Z], axis=0).transpose((1, 0))

geom = o3d.geometry.PointCloud()
geom.points = o3d.utility.Vector3dVector(point_data)
o3d.visualization.draw_geometries([geom])

In [ ]:
print("Point Cloud points: ", np.sum(indexes_needed))

In [ ]:
point_data

In [ ]:
gg = list(zip(x_values, y_values))

In [ ]:
print(gg)

In [ ]:

#point_data = np.stack([las.X, las.Y, las.Z], axis=0).transpose((1, 0))

#geom = o3d.geometry.PointCloud()
#geom.points = o3d.utility.Vector3dVector(point_data)
#o3d.visualization.draw_geometries([geom])